# HORTON3 Notebook

In [1]:
import numpy as np

## For Google Colab: Install Packages

In [ ]:
! pip install git+https://github.com/theochem/iodata.git
! pip install git+https://github.com/theochem/grid.git
! pip install git+https://github.com/theochem/gbasis.git
! pip install git+https://github.com/theochem/denspart.git

## For Goole Colab: Download Example file

In [ ]:
# download the example files
import os
from urllib.request import urlretrieve

fpath = "data/"
if not os.path.exists(fpath):
    os.makedirs(fpath, exist_ok=True)

urlretrieve(
    "https://raw.githubusercontent.com/theochem/horton3/master/notebooks/data/h2o_sto3g.fchk",
    os.path.join(fpath, "h2o_sto3g.fchk")
    )

## IOData

[GitHub Repository](https://github.com/theochem/iodata) | [Webpage](https://iodata.qcdevs.org/) | [Publication](https://onlinelibrary.wiley.com/doi/10.1002/jcc.26468)

In [2]:
from iodata import load_one

Check [IOData Webpage](https://iodata.qcdevs.org/) for list of supported file formats and properties.

In [3]:
mol = load_one("data/h2o_sto3g.fchk")

print("Number of atoms    = ", mol.natom)
print("Atomic numbers     = ", mol.atnums)
print("Atomic coordinates = ")
print(mol.atcoords)
print("")
print("MO Occupations = ", mol.mo.occs)
print("MO Energies    = ", mol.mo.energies)

Number of atoms    =  3
Atomic numbers     =  [8 1 1]
Atomic coordinates = 
[[-4.44734101  3.39697999  0.        ]
 [-2.58401495  3.55136194  0.        ]
 [-4.92380519  5.2049622   0.        ]]

MO Occupations =  [2. 2. 2. 2. 2. 0. 0.]
MO Energies    =  [-20.2515759   -1.25754909  -0.59385665  -0.45972894  -0.39261675
   0.58179435   0.69267699]


## Grid

[GitHub Repository](https://github.com/theochem/grid) | [Webpage](https://grid.qcdevs.org/)

In [4]:
from grid.becke import BeckeWeights
from grid.molgrid import MolGrid
from grid.onedgrid import GaussChebyshev
from grid.rtransform import BeckeRTransform

In [5]:
# Make molecular grid (using grid package)
oned = GaussChebyshev(100)
rgrid = BeckeRTransform(1e-4, 1.5).transform_1d_grid(oned)
grid = MolGrid.from_size(mol.atnums, mol.atcoords, rgrid, 110, BeckeWeights())

print("Grid points shape  = ", grid.points.shape)
print("Grid weights shape = ", grid.weights.shape)

Grid points shape  =  (33000, 3)
Grid weights shape =  (33000,)


## GBasis

[GitHub Repository](https://github.com/theochem/gbasis)

In [6]:
from gbasis.wrappers import from_iodata
from gbasis.evals.density import evaluate_density

In [7]:
# Compute molecular density (using gbasis package)
one_rdm = mol.one_rdms.get("post_scf", mol.one_rdms.get("scf"))
if one_rdm is None:
    if mol.mo is None:
        raise ValueError(
            "The input file lacks wavefunction data with which "
            "the density can be computed."
        )
    coeffs, occs = mol.mo.coeffs, mol.mo.occs
    one_rdm = np.dot(coeffs * occs, coeffs.T)
basis = from_iodata(mol)
density = evaluate_density(one_rdm, basis[0], grid.points, coord_type=basis[1])

print("Density shape = ", density.shape)

Density shape =  (33000,)


In [8]:
# Integrate electron density
print("Integral Density = ", grid.integrate(density))

Integral Density =  10.000088051536862


## Denspart

[GitHub Repository](https://github.com/theochem/denspart)

In [9]:
from denspart.mbis import MBISProModel
from denspart.vh import optimize_reduce_pro_model
from denspart.properties import compute_radial_moments, compute_multipole_moments, safe_ratio

For details, check [MBIS Method (Publication)](https://pubs.acs.org/doi/abs/10.1021/acs.jctc.6b00456).

In [12]:
# MBIS partitioning (using denspart package)
pro_model_init = MBISProModel.from_geometry(mol.atnums, mol.atcoords)

pro_model, localgrids = optimize_reduce_pro_model(
    pro_model_init,
    grid,
    density,
    # args.gtol,
    # args.maxiter,
    # args.density_cutoff,
)
print("Promodel")
pro_model.pprint()

print("Computing additional properties")
results = pro_model.to_dict()

results.update(
    {
        "charges": pro_model.charges,
        "radial_moments": compute_radial_moments(
            pro_model, grid, density, localgrids
        ),
        "multipole_moments": compute_multipole_moments(
            pro_model, grid, density, localgrids
        ),
        # "gtol": args.gtol,
        # "maxiter": args.maxiter,
        # "density_cutoff": args.density_cutoff,
    }
)
# np.savez_compressed(args.out_npz, **results)
print("")
print("Atomic Charges    = ", results["charges"])
print("")
print("Atomic Multipoles = ", results["multipole_moments"])

Building local grids
Integral of density: 10.000088051536862
Optimization
#Iter  #Call         ekld          kld  -constraint     grad.rms  cputime (s)
-----  -----  -----------  -----------  -----------  -----------  -----------
    1      1    0.5936396    0.5937277  -8.8052e-05   2.4770e-01    0.0184420
    2      6    0.3753745    0.2911400   8.4234e-02   1.2533e-01    0.0131760
    3     12    0.2686959    0.1705651   9.8131e-02   6.2372e-02    0.0131140
    4     12    0.2686959    0.1705651   9.8131e-02   6.2372e-02    0.0131140
    5     19    0.2305333    0.1914995   3.9034e-02   1.2143e-02    0.0138760
    6     19    0.2305333    0.1914995   3.9034e-02   1.2143e-02    0.0138760
    7     28    0.2244087    0.2004450   2.3964e-02   2.7907e-03    0.0129970
    8     28    0.2244087    0.2004450   2.3964e-02   2.7907e-03    0.0129970
    9     35    0.2242477    0.2167734   7.4743e-03   7.5061e-04    0.0141250
   10     35    0.2242477    0.2167734   7.4743e-03   7.5061e-04    